In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from itertools import combinations

In [98]:
def randpoints(n, scale=1):
    xp = np.random.random(n) * scale
    xp = np.append(xp,xp[0]) # need duplicate points for periodic boundary
    return xp

def randthetas(n): # Not used at the moment
    """ Ensures that we cover the whole range."""
    thetas = randpoints(n, scale=2*np.pi)
    thetas[0] = 0
    thetas[-1] = 2*np.pi
    return thetas


def halve(ls):
    """ Splits a list of two sublists in half"""
    ls1 = ls[0][len(ls[0])//2:] ,(ls[1][len(ls[1])//2:])
    ls2 = ls[0][:len(ls[0])//2] ,(ls[1][:len(ls[1])//2])
    return ls1,ls2

def breakup(ls,order):
    """Breaks up ls into 2^order sublists"""
    if order == 1: # base case
        return halve(ls)
    else:
        ls1, ls2 = halve(ls)
        return breakup(ls1,order-1) + breakup(ls2,order-1)

def range_intersects(rng1,rng2):
    """ rng = (xmin, xmax)"""
    return (rng1[0] <= rng2[0] <= rng1[1] or 
            rng2[0] <= rng1[0] <= rng2[1] or
            rng1[0] <= rng2[1] <= rng1[1] or
            rng2[0] <= rng1[1] <= rng2[1])    
    
def box_intersects(box1,box2):
    """ box = (xmin,xmax,ymin,ymax)"""
    return (range_intersects((box1[0],box1[1]),(box2[0],box2[1])) and
            range_intersects((box1[2],box1[3]),(box2[2],box2[3]))  )

def curve_intersects_rec(c1,c2, thresh):
    if len(c1[0]) <= thresh: # base case
        return True
    # construct bounding boxes
    box1 = (np.min(c1[0]),np.max(c1[0]),np.min(c1[1]),np.max(c1[1]))
    box2 = (np.min(c2[0]),np.max(c2[0]),np.min(c2[1]),np.max(c2[1]))
    
    if SHOW_STEPS:
        plt.figure()
        plt.axis((-1,2,-1,2))
        plt.plot(c1[0],c1[1],c2[0],c2[1])
        plt.title(str(box_intersects(box1,box2)) + ' size =' + str(len(c1[0])))
        plt.show()
    
    if box_intersects(box1,box2): #split the curves in half, recurse
        c1a, c1b = halve(c1)
        c2a, c2b = halve(c2)
        return (curve_intersects_rec(c1a,c2a,thresh) or 
                curve_intersects_rec(c1a,c2b,thresh) or
                curve_intersects_rec(c1b,c2a,thresh) or
                curve_intersects_rec(c1b,c2b,thresh) )
    else:
        return False

def curve_intersects(c, thresh = 100):
    """ Takes as input two curves c1 = [x,y]
    Returns True if c1 and c2 intersect. 
    Works by recursing on bounding boxes.
    Thanks to the lovely Pomax for the method."""
    assert len(c[0]) == len(c[1])
    assert len(c[0]) > 10 # it'll give true by default if you start with a small list

    # Hacky fix - some self-intersections get lost if you don't break it up enough
    cs = breakup(c,3)
    c_pairs = combinations(cs,r=2) # try each combination

    if SHOW_FAILS:
        plt.figure()
        plt.axis((-0.5,1.5,-0.5,1.5))
        for curve in cs:
            plt.plot(curve[0],curve[1], thresh)
        plt.show()
    
    for pair in c_pairs:
        if curve_intersects_rec(pair[0],pair[1], thresh):
            return True
       
    if SHOW_WINS:
        plt.figure()
        for curve in cs:
            plt.plot(curve[0],curve[1])
        plt.show()

    return False

def interp(points, n = 2000):
    """Takes as input list points = (x,y)
    returns a list [xnew,ynew] of interpolated points of length n.
    n is only matters for curve_intersects, n=2000 seems to work."""
    tck, u = interpolate.splprep(points, s=0, per=True)
    unew = np.linspace(0,1,n)
    xnew,ynew = interpolate.splev(unew, tck)
    xnew = np.delete(xnew,0) # need to remove duplicate points
    ynew = np.delete(ynew,0)
    return xnew,ynew

def make_random_shape(n_pts):
    """ Interpolate a random shape out of n_pts starting points.
    Starts to take way too long for n_pts > 8
    """
    valid_shape = False
    failcounter = 0
    while not valid_shape:
        points = randpoints(n_pts), randpoints(n_pts)
        fit_pts = interp(points)
        if curve_intersects(fit_pts):
            failcounter += 1
            continue
        valid_shape = True
#         print str(failcounter) + ' fails'
        return fit_pts

def make_shape(pts, output_len = 100):
    """ 
    Args:
        pts: a tuple of points (x,y) to be interpolated
        output_len: the number of points in the interpolated curve

    Returns:
        tuple: the pair of interpolated points (xnew,ynew)
    
    Raises:
        ValueError: pts defined a self-intersecting curve
    """
    assert len(pts[0]) == len(pts[1])
    fit_pts = interp(pts)
    if curve_intersects(fit_pts):
        raise ValueError("Curve is self-intersecting")
    sparse_pts = map(lambda ls:ls[::len(fit_pts[0])//output_len], fit_pts)
    print(len(fit_pts[0])//output_len)
    return sparse_pts

In [99]:
RAND_POINTS = 6
SHOW_STEPS = False
SHOW_WINS = True
SHOW_FAILS = False
VIEWS = 3


In [101]:
if __name__ == "__main__":
    views = 0
    while views < VIEWS:
        randcurve = make_random_shape(RAND_POINTS)
        print randcurve[::100]
        views+=1
#     print(len(make_shape(([1,2,3,4,3,0],[2,4,3,1,1,0]) , output_len = 500 )[0]))

(array([ 0.36176557,  0.36270635,  0.36365107, ...,  0.35896706,
        0.35989593,  0.36082876]),)
(array([ 0.30988839,  0.3094105 ,  0.30893566, ...,  0.31134101,
        0.31085362,  0.31036942]),)
(array([ 0.86931134,  0.8683936 ,  0.86747296, ...,  0.87204699,
        0.87113805,  0.87022616]),)
